In [ ]:
!git clone https://github.com/0601p/Model-Compression.git

In [ ]:
!pip install torch-pruning # v1.0.0

Code for Pruning

In [ ]:
import torch
import torch.nn.utils.prune as prune
import torchvision
import random
import sys
import torch_pruning
sys.path.append("/content/Model-Compression")

from model import *

device = 'cuda' if torch.cuda.is_available() else 'cpu'
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed(777)

In [ ]:
learning_rate = 1e-2
batch_size = 512
epoch = 20

In [ ]:
root = "./CIFAR100"
transform = torchvision.transforms.ToTensor()
data_train = torchvision.datasets.CIFAR100(root, train = True, transform = transform, download = True)
data_test = torchvision.datasets.CIFAR100(root, train = False, transform = transform, download = True)

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=data_train, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = torch.utils.data.DataLoader(dataset=data_test, batch_size=batch_size, shuffle=False, drop_last=True)

In [ ]:
model = ResNet(Bottleneck, [3, 4, 6, 3], num_classes=100).to(device)
checkpoint = torch.load("/content/Model-Compression/model.pth")
model.load_state_dict(checkpoint)
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [ ]:
DG = torch_pruning.DependencyGraph()
DG.build_dependency(model, example_inputs=torch.randn(1, 3, 32, 32))
pruning_idxs = [2, 6, 9]
pruning_group = DG.get_pruning_group(model.conv1, torch_pruning.prune_conv_out_channels, idxs = pruning_idxs)

if DG.check_pruning_group(pruning_group):
    pruning_group.exec()

Code for Training and Eval

In [ ]:
def train_one_epoch(print_result = False):
    model.train()
    loss_sum = 0.0
    accuracy_sum = 0.0
    length = 0

    for X, Y in train_loader:
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()

        pred = model(X)
        loss = criterion(pred, Y)
        pred_idx = torch.argmax(pred, 1)
        loss_sum += loss.item()
        accuracy_sum += torch.sum((pred_idx == Y).float()).item()
        length += X.size(0)

        loss.backward()
        optimizer.step()

    if(print_result):
        print("loss :", loss_sum / length)
        print("accuracy:", accuracy_sum / length)

In [ ]:
def eval():
    with torch.no_grad():
        model.eval()
        loss_sum = 0.0
        accuracy_sum = 0.0
        length = 0

        for X, Y in train_loader:
            X = X.to(device)
            Y = Y.to(device)

            pred = model(X)
            loss = criterion(pred, Y)
            pred_idx = torch.argmax(pred, 1)
            loss_sum += loss.item()
            accuracy_sum += torch.sum((pred_idx == Y).float()).item()
            length += X.size(0)
            
        print("loss :", loss_sum / length)
        print("accuracy:", accuracy_sum / length)

In [ ]:
def train():
    for i in range(epoch):
        print("EPOCH[" + str(i + 1) + "]")
        print("==== train ====")
        train_one_epoch(print_result=True)
        
        print("==== eval ====")
        eval()

In [ ]:
eval()